<a href="https://colab.research.google.com/github/mattdoesstuff/covid/blob/master/COVID_19_vs_Data_Science_Legends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Svanik Dani & Mathias Prodani

In [0]:
import pandas as pd
import numpy as np
import altair as alt
from datetime import datetime, timedelta

John Hopkins warning in their Repo:

---DEPRICATED WARNING---
The files below will no longer be updated. With the release of the new data structure, we are updating our time series tables to reflect these changes. Please reference time_series_covid19_confirmed_global.csv and time_series_covid19_deaths_global.csv for the latest time series data.

time_series_19-covid-Confirmed.csv
time_series_19-covid-Deaths.csv
time_series_19-covid-Recovered.csv

In [0]:
#Create a DATELIST to be the same as the dates in the dataset
start = datetime.strptime("22-01-2020", "%d-%m-%Y")
end = datetime.today()
DATELIST = [start + timedelta(days=x) for x in range(0, (end-start).days)]

In [0]:
#Read in datasets
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")

In [0]:
# Combine all the province separated data into by country data
confirmed = confirmed.groupby(confirmed['Country/Region'])[confirmed.columns].sum()
deaths = deaths.groupby(deaths['Country/Region'])[deaths.columns].sum()

In [0]:
# Sort by the data in the most recent day
confirmed.sort_values(by=confirmed.columns[-1], ascending=False, inplace=True)
deaths.sort_values(by=confirmed.columns[-1], ascending=False, inplace=True)

In [0]:
# Take top 30 countries in confirmed deaths and confirmed cases
confirmedStripped = confirmed[:30]
deathsStripped = deaths[:30]

In [0]:
# Get countries that are top in both deaths and confirmed
top30con = confirmedStripped.index
top30deaths = deathsStripped.index

topcountries = []
for i in top30con:
    if i in top30deaths:
        topcountries.append(i)

In [0]:
# Create column vectors of each for in the top 30
confirmedColVecs = [] # [['China', [*LIST OF COUNTS*]]]
for i in range(len(confirmedStripped)):
    subset = confirmedStripped[i:i+1]
    subset = subset.transpose()
    confirmedColVecs.append([subset.columns[0], subset[subset.columns[0]][2:].values])

deathsColVecs = []
for i in range(len(deathsStripped)):
    subset = deathsStripped[i:i+1]
    subset = subset.transpose()
    deathsColVecs.append([subset.columns[0], subset[subset.columns[0]][2:].values])



In [0]:
# Make arrays of datasets for each country's deaths and cases
confirmedDatasets = []
for i in confirmedColVecs:
  confirmedDatasets.append([i[0], pd.DataFrame({
    'Date': DATELIST,
    'Cases': i[1]
  })])
deathDatasets = []
for i in deathsColVecs:
  deathDatasets.append([i[0], pd.DataFrame({
    'Date': DATELIST,
    'Deaths': i[1]
  })])

# Make list of death graphs
deathGraphs = []
for i in deathDatasets:
    # Advisory: Index title for death graphs and confirmed cases graphs is different
    c1 = alt.Chart(i[1]).mark_line().encode(
        x = 'Date:T',
        y = 'Deaths:Q'
    ).properties(title = i[0] + " Deaths over Time")
    deathGraphs.append(c1)

# Make list of confirmed graphs
confirmedGraphs = []
for i in confirmedDatasets:
    # Advisory: Index title for death graphs and confirmed cases graphs is different
    c1 = alt.Chart(i[1]).mark_line().encode(
        x = 'Date:T',
        y = 'Cases:Q'
    ).properties(title = i[0] + " Confirmed Cases over Time")
    confirmedGraphs.append(c1)

In [0]:
# Puts countries with both top 30 confirmed cases and deaths into a colVec format
combinedColVecs = {}
for i in deathsColVecs:
  if i[0] in topcountries:
    combinedColVecs[i[0]] = [i[1]]
combinedColVecs
for i in confirmedColVecs:
  if i[0] in topcountries:
    combinedColVecs[i[0]].append(i[1])

# Makes dict of dataframes for each country in both with deaths, confirmed cases, and deathrate
combinedDataFrames = {}
deathRateGraphs = {}
for key in combinedColVecs.keys():
  data = {"Date" : DATELIST, "Deaths": combinedColVecs[key][0], "Confirmed": combinedColVecs[key][1]}
  df = pd.DataFrame(data)
  df["Deathrate"] = df.Deaths.div(df.Confirmed, fill_value = 0)*100

  # Make graph
  graph = alt.Chart(df).mark_line().encode(
    x = 'Date:T',
    y = 'Deathrate:Q'
  ).properties(title = key + " Death Rate over Time (%)")
  deathRateGraphs[key] = graph
  combinedDataFrames[key] = df


In [0]:
# Compile Confirmed Cases Graphs into One
confirmedFig = confirmedGraphs.pop(0)
for i in range(len(confirmedGraphs)):
    confirmedFig = alt.vconcat(confirmedFig, confirmedGraphs[i])
confirmedFig

alt.VConcatChart(...)

In [0]:
#Compile deathgraphs in a figure
deathFig = deathGraphs.pop(0)
for i in range(len(deathGraphs)):
    deathFig = alt.vconcat(deathFig, deathGraphs[i])
deathFig


alt.VConcatChart(...)

In [0]:
# Compile Death Rate Graphs into One
deathRateFig = deathRateGraphs.pop(list(deathRateGraphs.keys())[0])
for key in deathRateGraphs.keys():
    deathRateFig = alt.vconcat(deathRateFig, deathRateGraphs[key])
deathRateFig

alt.VConcatChart(...)

### Reflections
__________________________________
It becomes clear form the graphs that the spread of COVID-19 follows the pattern of a logistic curve. Furthermore, it is possible to strech out the curve over time and lower the rate at which the virus spread through practice such as social distancing. Death rates vary significantly between countries: possible factors may include populations, average age, health care system, availability of testing, and the implementation of preventative measures. The death rates also vary signifacntly over time, possible factors for this may be, overwhelming of the health care system, lack of health care supplies, and lack of proper testing equipment. In the case of Italy these factors have lead to an upward trend. In the United States we see a sharp drop in the death rate after the implemntation of wide spread testing, leading to a more accurate count of cases and therefore a value closer to the mortality of the disease.

##### Biggest Challenge
________________________________________________________________________
Perhaps the biggest challenge was getting the time series format given by John Hopkins to get along with Altair and other libraries. A lot of data manipulation was required to get everything working as it should. The separated data will also allow us to perform techniques such as regression to come up with predictions. We will be working on this in the near future. Another issue we saw is that in the beginning we used arrays to create our data structures while towards the end, the use of dictionaries made it easier to manipulate.

MatPlotLib Animated Cases Overtime

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
fig, ax = plt.subplots(figsize=(15, 8))
def barchart(currentDate):
  currentDate = DATELIST[currentDate].strftime("%-m/%-d/%y")
  df = confirmed[[currentDate]]
  df = df.sort_values(by= currentDate, ascending = False).head(10).reset_index()
  ax.clear()
  ax.barh(df["Country/Region"], df[currentDate])

In [0]:

animator = animation.FuncAnimation(fig, barchart, frames=range(len(DATELIST)))
# animator.save(filename="fig.mp4", fps=5)
HTML(animator.to_jshtml())